In [2]:
import collections
import numpy as np
import pandas
import pickle
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import torch.nn.functional as F
import math
from metrics import Metrics
import key_tools as tools

# Some magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [3]:
def LoadData(filename):
    data = pandas.read_csv(filename, header=None, sep='\t', quoting=3)
    data.columns = ['lang', 'text']
    return data
data = LoadData('../Data/train.tsv')
val_data = LoadData('../Data/val.tsv')
test_data = LoadData('../Data/test.tsv')
data.head()

,lang,text
0,es,"Alemania vs Argentina, la tercera es la vencid..."
1,en,I have gained level 39 in The Tribez and Castl...
2,pt,"Finalmente é sexta, mas ainda tenho teste"
3,fr,"""#Marée ↗ #Mimizan 14/04/2016 UTC+2 Basse mer ..."
4,es,Que ganitas de poder ir al gym jupe..


In [4]:
# Create the vocabulary tables
all_langs = data.lang.unique()
lang2idx = dict(zip(all_langs, range(len(all_langs))))
idx2lang = dict(zip(lang2idx.values(), lang2idx.keys()))

counts = collections.Counter()
for line in data.text:
    counts.update(line)

chars = set([c for c in counts if counts[c] >= 10])
chars.update(['PAD', '<S>', '</S>', 'UNK'])
char2idx = dict(zip(chars, range(len(chars))))
idx2char = dict(zip(range(len(chars)), chars))

In [5]:
# size of vocabulary
print('size of vocabulary is {0}'.format(len(chars)))
print('number of unique characters {0}'.format(len(counts)))

total_chars = sum(counts.values())
total_oovs = sum([counts[c] for c in counts if counts[c] < 10])
print('oov rate is {0:.4f}%'.format(100.0 * total_oovs / total_chars))

size of vocabulary is 510
number of unique characters 1393
oov rate is 0.0469%


In [6]:
# WARM_UP: Compute the perplexity of a unigram model
train_counts = np.zeros(len(chars))
val_counts = np.zeros(len(chars))

for line in data.text:
    for c in line:
        idx = char2idx.get(c, char2idx['UNK'])
        train_counts[idx] += 1.0
    train_counts[char2idx['</S>']] += 1.0
for line in val_data.text:
    for c in line:
        idx = char2idx.get(c, char2idx['UNK'])
        val_counts[idx] += 1.0
    val_counts[char2idx['</S>']] += 1.0
train_counts[char2idx['PAD']] += 1
train_counts = train_counts / train_counts.sum()
train_counts[char2idx['<S>']] = 1.0  # this will be zeroed out later
val_counts = val_counts / val_counts.sum()
ppl = np.exp(-(val_counts * np.log(train_counts)).sum())
print('the perplexity is {0:.2f}'.format(ppl))

the perplexity is 34.11


In [7]:
def prepare_data(char2idx, idx2char, lang2idx, idx2lang, data, save_file, max_seq_length=1000):
    sequences = []
    languages = []
    for i in range(len(data.text)):
        line = data.text[i]
        if len(line) > max_seq_length:
            continue
        seq = [char2idx['<S>']]
        for c in line:
            idx = char2idx.get(c, char2idx['UNK'])
            seq += [idx]
        seq += [char2idx['</S>']]
        sequences += [seq]
        languages += [lang2idx[data.lang[i]]]

    pickle.dump({'chars': sequences, 'langs': languages, 'ind2voc': idx2char, 'voc2ind':char2idx, 'ind2lang': idx2lang, 'lang2ind':lang2idx}, open(save_file, 'wb'))
    
prepare_data(char2idx, idx2char, lang2idx, idx2lang, data, 'chars_train.pkl')
prepare_data(char2idx, idx2char, lang2idx, idx2lang, val_data, 'chars_val.pkl')
prepare_data(char2idx, idx2char, lang2idx, idx2lang, test_data, 'chars_test.pkl')


PAD = char2idx['PAD']

In [62]:
class MyRNN(nn.Module):
    def __init__(self, vocab_size, lang_size=9, char_vec_size=12, lang_vec_size=2, hidden_size=50, PAD=0):
        
        super(MyRNN, self).__init__()
        self.vocab_size = vocab_size
        self.lang_size = lang_size
        self.char_vec_size = char_vec_size
        self.lang_vec_size = lang_vec_size
        self.hidden_size = hidden_size
        
        self.char_encoder = nn.Embedding(self.vocab_size, self.char_vec_size)
        self.lang_encoder = nn.Embedding(self.lang_size, self.lang_vec_size)
        # the current hidden size = char_vec_size
        self.gru = nn.GRU(self.char_vec_size+self.lang_vec_size, self.hidden_size, num_layers=1)
        self.linear = nn.Linear(self.hidden_size, self.char_vec_size)
        self.decoder = nn.Linear(self.char_vec_size, self.vocab_size)
        
        # This shares the encoder and decoder weights as described in lecture.
        self.decoder.weight = self.char_encoder.weight
        self.decoder.bias.data.zero_()
        
        
        weight = torch.ones(vocab_size)
        # scores over PAD is not counted
        weight[PAD] = 0
        self.sm = nn.LogSoftmax(dim=1)
        self.crit = nn.NLLLoss(weight, size_average=False)

    def forward(self, input, hidden=None):
        emb = pack(torch.cat((self.char_encoder(input[0]), self.lang_encoder(input[1])), -1), input[2])
        output, hidden_t = self.gru(emb, hidden)
        output = unpack(output)[0]
        output = F.tanh(self.linear(output))
        output = self.decoder(output)
        return output, hidden_t

    # Predefined loss function
    def loss(self, prediction, label, reduction='elementwise_mean'):
        prediction = prediction.view(-1, self.vocab_size)
        prediction = self.sm(prediction)
        loss_val = self.crit(prediction, label.view(-1))
        return loss_val

In [63]:
verbose = False

BATCH_SIZE = 64
FEATURE_SIZE = 15
TEST_BATCH_SIZE = 256
EPOCHS = 10
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0005

data_train = tools.Dataset('chars_train.pkl', BATCH_SIZE, PAD)
data_val = tools.Dataset('chars_val.pkl', TEST_BATCH_SIZE, PAD)
data_test = tools.Dataset('chars_test.pkl', TEST_BATCH_SIZE, PAD)

model = MyRNN(len(char2idx),PAD=PAD)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [88]:
for epoch in range(EPOCHS):
    print(epoch+1)
    train_ppl = tools.train(model, data_train, epoch, optimizer, PAD)
    val_loss, val_ppl = tools.test(model, data_val, PAD)
predictions = tools.get_predictions(model, data_test)

1
epoch : 0
     train_ppl : 33.83951354630192
     train_loss : 13687.712269459547
test_ppl : 17.229011850219806
test_loss : 47336.68512228261
2
epoch : 1
     train_ppl : 15.78197560253124
     train_loss : 11452.440121053221
test_ppl : 14.694313372765583
test_loss : 44689.497027853264
3
epoch : 2
     train_ppl : 14.199412668699447
     train_loss : 11016.949385537211
test_ppl : 13.69183769980953
test_loss : 43514.11948029891
4


KeyboardInterrupt: 

In [ ]:
Metrics(predictions, data_test.langs)